In [1]:
import numpy as np
import pandas as pd
from src.utils import * 
from src.metrics_FAMD import *

In [2]:
gbsg = pd.read_csv("../GBSG2")

# On a un jeu de données complet: 
np.where(gbsg.isna()==True)

(array([], dtype=int64), array([], dtype=int64))

In [3]:
n_it = 100

On détermine $I$ et $J$: 

In [4]:
I, J = gbsg.shape

Trouver $K_1$, $K_2$

In [5]:
# Variables continues
idx_k1 = gbsg.columns[(gbsg.isin([0, 1])==False).any()]

In [6]:
for i in gbsg.columns[(gbsg.isin([0, 1])).all()].to_numpy(): 
    gbsg[i+'_'] = 1-gbsg[i]

#Variables encodées en dummy variables 
idx_j = gbsg.columns[(gbsg.isin([0, 1])).all()]

In [7]:
# we compute the missingness on categories before being processed
proba_non_missing = 0.8

data_missing_without_last_cat = gbsg.iloc[:,:-3].mask(np.random.random(size=(gbsg.iloc[:,:-3].shape))>proba_non_missing)
data_missing_last_cat = gbsg[['status_', 'meno_', 'hormon_']].mask(data_missing_without_last_cat[['status', 'meno', 'hormon']].isna().to_numpy())
data_missing = pd.concat((data_missing_without_last_cat, data_missing_last_cat), axis=1)

In [8]:
n_missing = data_missing.isna().sum().sum()
n_missing/(data_missing.shape[0]*data_missing.shape[1])

0.20501874219075386

In [9]:
famd_algo = FAMD(data=gbsg, k1=idx_k1, k2=idx_j)
# famd_algo.run_famd()

In [10]:
ifamd = IterativeFAMDImputer(n_components=4, data=data_missing, k1=idx_k1, k2=idx_j)
ifamd.impute(100)

Maximum iterations reached


,pid,age,size,grade,nodes,pgr,er,rfstime,meno,hormon,status,meno_,hormon_,status_
0,132.000000,49.000000,18.000000,2.000000,2.000000,0.000000,0.000000,1838.000000,0.539646,0.430414,0.462191,0.460354,0.569586,0.537809
1,890.488175,55.000000,20.000000,3.000000,16.000000,0.000000,0.000000,1133.446691,0.527949,0.338618,0.453466,0.000000,0.661773,0.545994
2,1140.000000,56.000000,40.000000,3.000000,5.193487,40.164059,82.172385,1133.446691,1.000000,0.347403,0.000000,0.000000,0.652584,1.000000
3,769.000000,45.000000,35.186228,2.124304,7.321847,0.000000,4.000000,177.000000,0.539625,0.000000,0.000000,0.460375,0.706759,1.000000
4,979.402892,65.000000,30.000000,2.000000,5.193487,35.512569,330.001071,1769.810425,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,970.001148,51.000000,30.000000,3.000000,2.000000,1152.000000,38.000000,1760.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
682,1273.000000,64.000000,26.000000,2.124304,4.626638,1356.000000,1144.000000,1133.446691,1.237794,0.849855,-0.058625,-0.234915,0.000000,1.059886
683,1525.000000,57.000000,35.000000,3.000000,1.000000,1490.000000,93.651685,1048.443353,1.000000,0.346310,0.000000,0.000000,0.000000,1.000000
684,736.000000,33.250928,30.877934,1.852972,3.000000,1600.000000,70.000000,629.000000,0.539625,0.000000,0.000000,0.460375,0.604567,1.000000


#### To ensure that we are only considering TPR and NRMSE scores on imputed data: 

In [11]:
C0_missing = data_missing.isna()[ifamd.k1].to_numpy()
Categ_missing = data_missing.isna()[ifamd.k2].to_numpy()

#### Falsely classified Rate: 

In [12]:
# We encode categories into 0,1
res = (ifamd.df[ifamd.k2].copy()>=0.5).astype(int)

metric_fc(res[Categ_missing], gbsg[ifamd.k2][Categ_missing])

array([0.31449631, 0.28255528, 0.19410319, 0.35135135, 0.28255528,
       0.21375921])

#### NRMSE

In [13]:
compute_nrmse_weighted(gbsg[ifamd.k1][C0_missing], ifamd.df[ifamd.k1][C0_missing])

19.91518504327409